In [ ]:
#!/usr/bin/python3
# coding: utf-8
# sapporo

In [ ]:
import codecs
import sys
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
from urllib.request import urlretrieve
from cov19utils import download_if_needed